In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [93]:
#api_key = "88382916-d527-45b1-bd27-ccce14a9bac6"

url = "https://api.pokemontcg.io/v2/cards"
#headers = {"X-Api-Key": api_key}
page = 1
all_cards = []

for i in range(1):
    params = {"page": page, "pageSize": 250} 
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()["data"]
        if not data:
            break 
        all_cards.extend(data)
        print(data)
        page += 1  
    else:
        print(f"Erreur: {response.status_code}")
        break

print(f"Nombre total de cartes récupérées : {len(all_cards)}")

for card in all_cards[:10]:
    print(f"Nom de la carte: {card['name']}, Set: {card['set']['name']}")


[{'id': 'dp3-1', 'name': 'Ampharos', 'supertype': 'Pokémon', 'subtypes': ['Stage 2'], 'level': '52', 'hp': '130', 'types': ['Lightning'], 'evolvesFrom': 'Flaaffy', 'abilities': [{'name': 'Jamming', 'text': "After your opponent plays a Supporter card from his or her hand, put 1 damage counter on each of your opponent's Pokémon. You can't use more than 1 Jamming Poké-Body each turn.", 'type': 'Poké-Body'}], 'attacks': [{'name': 'Cluster Bolt', 'cost': ['Lightning', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '70', 'text': "You may discard all Lightning Energy attached to Ampharos. If you do, this attack does 20 damage to each of your opponent's Benched Pokémon that has any Energy cards attached to it. (Don't apply Weakness and Resistance for Benched Pokémon.)"}], 'weaknesses': [{'type': 'Fighting', 'value': '+30'}], 'resistances': [{'type': 'Metal', 'value': '-20'}], 'retreatCost': ['Colorless', 'Colorless', 'Colorless'], 'convertedRetreatCost': 3, 'set': {'id': 'dp3',

In [96]:
df_messy

,id,name,supertype,subtypes,level,hp,types,evolvesFrom,abilities,attacks,...,legalities,images,tcgplayer,cardmarket,evolvesTo,rules,regulationMark,url,holofoil_price,reverse_holofoil_price
0,dp3-1,Ampharos,Pokémon,[Stage 2],52,130,[Lightning],Flaaffy,"[{'name': 'Jamming', 'text': 'After your oppon...","[{'name': 'Cluster Bolt', 'cost': ['Lightning'...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/dp3/1....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,NaN,14.02,11.17
1,ex12-1,Aerodactyl,Pokémon,[Stage 1],NaN,70,[Colorless],Mysterious Fossil,"[{'name': 'Reactive Protection', 'text': 'Any ...","[{'name': 'Power Blow', 'cost': ['Colorless'],...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/ex12/1...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,NaN,20.76,32.49
2,mcd19-1,Caterpie,Pokémon,[Basic],NaN,50,[Grass],NaN,NaN,"[{'cost': ['Grass'], 'name': 'Surprise Attack'...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/mcd19/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,[Metapod],NaN,NaN,NaN,NaN,NaN
3,ex7-1,Azumarill,Pokémon,[Stage 1],NaN,80,[Water],Marill,"[{'name': 'Froth', 'text': 'Once during your t...","[{'name': 'Water Punch', 'cost': ['Water', 'Co...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/ex7/1....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,NaN,24.88,15.66
4,sm9-1,Celebi & Venusaur-GX,Pokémon,"[Basic, TAG TEAM, GX]",NaN,270,[Grass],NaN,NaN,"[{'name': 'Pollen Hazard', 'cost': ['Grass', '...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/sm9/1....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,[TAG TEAM rule: When your TAG TEAM is Knocked ...,NaN,NaN,11.65,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,sm12-4,Vileplume-GX,Pokémon,"[Stage 2, GX]",NaN,240,[Grass],Gloom,"[{'name': 'Fragrant Flower Garden', 'text': 'O...","[{'name': 'Massive Bloom', 'cost': ['Grass', '...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/sm12/4...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,[Pokémon-GX rule: When your Pokémon-GX is Knoc...,NaN,NaN,NaN,NaN
246,xy5-6,Treecko,Pokémon,[Basic],NaN,60,[Grass],NaN,NaN,"[{'name': 'Quick Attack', 'cost': ['Grass'], '...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/xy5/6....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,[Grovyle],NaN,NaN,NaN,NaN,NaN
247,mcd19-7,Gastly,Pokémon,[Basic],NaN,50,[Psychic],NaN,NaN,"[{'cost': ['Psychic'], 'name': 'Ominous Eyes',...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/mcd19/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,[Haunter],NaN,NaN,NaN,NaN,NaN
248,gym2-5,Giovanni's Gyarados,Pokémon,[Stage 1],40,90,[Water],Giovanni's Magikarp,NaN,"[{'name': 'Summon Storm', 'cost': ['Water'], '...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/gym2/5...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
x=df["tcgplayer"][1]

{'url': 'https://prices.pokemontcg.io/tcgplayer/ex12-1',
 'updatedAt': '2024/11/20',
 'prices': {'holofoil': {'low': 13.02,
   'mid': 17.99,
   'high': 39.99,
   'market': 20.76,
   'directLow': 39.99},
  'reverseHolofoil': {'low': 17.0,
   'mid': 32.99,
   'high': 99.99,
   'market': 32.49,
   'directLow': 99.99}}}

In [61]:
def filter_holofoil_data(x):
    prices=x["prices"]
    holofoil_data=prices["holofoil"]
    holofoil_price=holofoil_data["market"]
    reverse_holofoil_data=prices["reverseHolofoil"]
    reverse_holofoil_price = reverse_holofoil_data["market"]
    return x['url'], holofoil_price, reverse_holofoil_price


In [62]:
print(filter_holofoil_data(df["tcgplayer"][1]))

('https://prices.pokemontcg.io/tcgplayer/ex12-1', 20.76, 32.49)


In [127]:
variables_to_drop=['attacks',
                   'supertype',
                   'hp',
                   'abilities',
                   'types',
                   'level',
                   'evolvesFrom',
                   'evolvesTo',
                   'weaknesses',
                   'resistances',
                   'retreatCost',
                   'convertedRetreatCost',
                   'flavorText',
                   'legalities',
                    'images',
                    'rules',
                    'regulationMark',
                    'ancientTrait',
                    'number',
                    'set',
                    'subtypes',
                    'tcgplayer',
                    'cardmarket'
                    
]


def get_prices(x):
    if pd.isna(x["tcgplayer"]) or x["rarity"] == "Common" or pd.isna(x["rarity"]):
        return None, None, None
    
    prices = x["tcgplayer"].get("prices", {})
    holofoil_data = prices.get("holofoil", {})
    reverse_holofoil_data = prices.get("reverseHolofoil", {})

    holofoil_price = holofoil_data.get("market", None)
    reverse_holofoil_price = reverse_holofoil_data.get("market", None)

    if (holofoil_price is not None and holofoil_price > 10) or (reverse_holofoil_price is not None and reverse_holofoil_price > 10):
        return x["tcgplayer"].get("url", None), holofoil_price, reverse_holofoil_price
    else:
        return None, None, None
    
    
def filter_holofoil_data(df):
    df[["url", "holofoil_price", "reverse_holofoil_price"]] = df.apply(get_prices, axis=1, result_type="expand")
    df_cleaned = df.dropna(subset=["url"])
    df_cleaned = df_cleaned[(df_cleaned["rarity"] != "Common") & ((df_cleaned["holofoil_price"] > 10) | (df_cleaned["reverse_holofoil_price"] > 10))]
    print(df_cleaned["set"])
    df_cleaned["collection"]=df_cleaned["set"].apply(lambda x:x["name"])
    df_cleaned["series"]=df_cleaned["set"].apply(lambda x:x["series"])
    
    #drop all the variables we used or not
    df_cleaned = df_cleaned.drop(columns=variables_to_drop, errors='ignore')


    return df_cleaned

In [128]:
df_messy=pd.DataFrame(all_cards)
df=filter_holofoil_data(df_messy)
df

0      {'id': 'dp3', 'name': 'Secret Wonders', 'serie...
1      {'id': 'ex12', 'name': 'Legend Maker', 'series...
3      {'id': 'ex7', 'name': 'Team Rocket Returns', '...
4      {'id': 'sm9', 'name': 'Team Up', 'series': 'Su...
5      {'id': 'pl1', 'name': 'Platinum', 'series': 'P...
                             ...                        
218    {'id': 'dp3', 'name': 'Secret Wonders', 'serie...
220    {'id': 'ex8', 'name': 'Deoxys', 'series': 'EX'...
227    {'id': 'ex14', 'name': 'Crystal Guardians', 's...
238    {'id': 'sm75', 'name': 'Dragon Majesty', 'seri...
249    {'id': 'base4', 'name': 'Base Set 2', 'series'...
Name: set, Length: 64, dtype: object


,id,name,artist,rarity,nationalPokedexNumbers,url,holofoil_price,reverse_holofoil_price,collection,series
0,dp3-1,Ampharos,Kouki Saitou,Rare Holo,[181],https://prices.pokemontcg.io/tcgplayer/dp3-1,14.02,11.17,Secret Wonders,Diamond & Pearl
1,ex12-1,Aerodactyl,Hajime Kusajima,Rare Holo,[142],https://prices.pokemontcg.io/tcgplayer/ex12-1,20.76,32.49,Legend Maker,EX
3,ex7-1,Azumarill,Sumiyoshi Kizuki,Rare Holo,[184],https://prices.pokemontcg.io/tcgplayer/ex7-1,24.88,15.66,Team Rocket Returns,EX
4,sm9-1,Celebi & Venusaur-GX,Mitsuhiro Arita,Rare Holo GX,"[3, 251]",https://prices.pokemontcg.io/tcgplayer/sm9-1,11.65,NaN,Team Up,Sun & Moon
5,pl1-2,Blastoise,Kagemaru Himeno,Rare Holo,[9],https://prices.pokemontcg.io/tcgplayer/pl1-2,20.01,12.34,Platinum,Platinum
...,...,...,...,...,...,...,...,...,...,...
218,dp3-5,Flygon,Daisuke Ito,Rare Holo,[330],https://prices.pokemontcg.io/tcgplayer/dp3-5,10.17,5.07,Secret Wonders,Diamond & Pearl
220,ex8-3,Breloom,Hajime Kusajima,Rare Holo,[286],https://prices.pokemontcg.io/tcgplayer/ex8-3,16.48,14.50,Deoxys,EX
227,ex14-2,Blastoise δ,Masakazu Fukuda,Rare Holo,[9],https://prices.pokemontcg.io/tcgplayer/ex14-2,51.70,85.76,Crystal Guardians,EX
238,sm75-3,Charizard,Ryota Murayama,Rare Holo,[6],https://prices.pokemontcg.io/tcgplayer/sm75-3,5.26,10.96,Dragon Majesty,Sun & Moon


In [122]:


df_messy["collection"]=df_messy["set"].apply(lambda x:x["name"])
df_messy["series"]=df_messy["set"].apply(lambda x:x["series"])
df_messy["tcg_url"] = df_messy["tcgplayer"].apply(lambda x: x['url'] if isinstance(x, dict) else None)



filtered_prices=filter_holofoil_data(df_messy["tcgplayer"])
print(filtered_prices)
#df_messy["tcg_url"]=filtered_prices["url"] if isinstance (filtered_prices["url"], dict) else None
df_messy["holofoil"]=filtered_prices["holofoil"]
df_messy["reverseholofoil"]=filtered_prices["reverseholofoil"]


df_messy["series"]=df_messy["set"].apply(lambda x:x["series"])
df_messy["series"]=df_messy["set"].apply(lambda x:x["series"])
df_messy["series"]=df_messy["set"].apply(lambda x:x["series"])
df_messy["series"]=df_messy["set"].apply(lambda x:x["series"])




df = df_messy.drop(columns=variables_to_drop, errors='ignore')

TypeError: get_prices() got an unexpected keyword argument 'axis'

In [40]:
df["tcgplayer"][1]

{'url': 'https://prices.pokemontcg.io/tcgplayer/ex12-1',
 'updatedAt': '2024/11/20',
 'prices': {'holofoil': {'low': 13.02,
   'mid': 17.99,
   'high': 39.99,
   'market': 20.76,
   'directLow': 39.99},
  'reverseHolofoil': {'low': 17.0,
   'mid': 32.99,
   'high': 99.99,
   'market': 32.49,
   'directLow': 99.99}}}

In [16]:
for x in df_messy["tcgplayer"]:
    print(x["url"])

https://prices.pokemontcg.io/tcgplayer/dp3-1
https://prices.pokemontcg.io/tcgplayer/ex12-1
https://prices.pokemontcg.io/tcgplayer/mcd19-1
https://prices.pokemontcg.io/tcgplayer/ex7-1
https://prices.pokemontcg.io/tcgplayer/sm9-1
https://prices.pokemontcg.io/tcgplayer/pl1-2
https://prices.pokemontcg.io/tcgplayer/ex3-2
https://prices.pokemontcg.io/tcgplayer/sm9-2
https://prices.pokemontcg.io/tcgplayer/mcd19-2
https://prices.pokemontcg.io/tcgplayer/xy5-2
https://prices.pokemontcg.io/tcgplayer/sm12-1
https://prices.pokemontcg.io/tcgplayer/si1-1
https://prices.pokemontcg.io/tcgplayer/sm3-1
https://prices.pokemontcg.io/tcgplayer/pop7-1
https://prices.pokemontcg.io/tcgplayer/bw1-2
https://prices.pokemontcg.io/tcgplayer/base4-3
https://prices.pokemontcg.io/tcgplayer/hgss4-3
https://prices.pokemontcg.io/tcgplayer/ex7-3
https://prices.pokemontcg.io/tcgplayer/swsh35-2
https://prices.pokemontcg.io/tcgplayer/xy2-3
https://prices.pokemontcg.io/tcgplayer/sm9-3
https://prices.pokemontcg.io/tcgplayer/sm

TypeError: 'float' object is not subscriptable

In [23]:
for x in df_messy['tcgplayer']:
    print(x)

{'url': 'https://prices.pokemontcg.io/tcgplayer/dp3-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 5.42, 'mid': 14.49, 'high': 44.99, 'market': 14.02, 'directLow': None}, 'reverseHolofoil': {'low': 4.27, 'mid': 7.99, 'high': 12.3, 'market': 11.17, 'directLow': None}}}
{'url': 'https://prices.pokemontcg.io/tcgplayer/ex12-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 13.02, 'mid': 17.99, 'high': 39.99, 'market': 20.76, 'directLow': 39.99}, 'reverseHolofoil': {'low': 17.0, 'mid': 32.99, 'high': 99.99, 'market': 32.49, 'directLow': 99.99}}}
{'url': 'https://prices.pokemontcg.io/tcgplayer/mcd19-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 4.44, 'mid': 5.17, 'high': 10.26, 'market': 5.19, 'directLow': None}}}
{'url': 'https://prices.pokemontcg.io/tcgplayer/ex7-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 14.11, 'mid': 22.39, 'high': 29.99, 'market': 24.88, 'directLow': 17.99}, 'reverseHolofoil': {'low': 15.94, 'mid': 25.0, 'hig

In [25]:
variables_to_drop=['attacks',
                   'supertype',
                   'hp',
                   'abilities',
                   'types',
                   'level',
                   'evolvesFrom',
                   'evolvesTo',
                   'weaknesses',
                   'resistances',
                   'retreatCost',
                   'convertedRetreatCost',
                   'flavorText',
                   'legalities',
                    'images',
                    'rules',
                    'regulationMark',
                    'ancientTrait',
                    'number'
]
df = df.drop(columns=variables_to_drop, errors='ignore')


In [26]:
df

,id,name,subtypes,set,number,artist,rarity,nationalPokedexNumbers,tcgplayer,cardmarket
0,dp3-1,Ampharos,[Stage 2],"{'id': 'dp3', 'name': 'Secret Wonders', 'serie...",1,Kouki Saitou,Rare Holo,[181],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
1,ex12-1,Aerodactyl,[Stage 1],"{'id': 'ex12', 'name': 'Legend Maker', 'series...",1,Hajime Kusajima,Rare Holo,[142],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
2,mcd19-1,Caterpie,[Basic],"{'id': 'mcd19', 'name': 'McDonald's Collection...",1,Sekio,NaN,[10],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
3,ex7-1,Azumarill,[Stage 1],"{'id': 'ex7', 'name': 'Team Rocket Returns', '...",1,Sumiyoshi Kizuki,Rare Holo,[184],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
4,sm9-1,Celebi & Venusaur-GX,"[Basic, TAG TEAM, GX]","{'id': 'sm9', 'name': 'Team Up', 'series': 'Su...",1,Mitsuhiro Arita,Rare Holo GX,"[3, 251]",{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
...,...,...,...,...,...,...,...,...,...,...
495,sm3-8,Tangrowth,[Stage 1],"{'id': 'sm3', 'name': 'Burning Shadows', 'seri...",8,tetsuya koizumi,Rare,[465],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
496,dp1-7,Luxray,[Stage 2],"{'id': 'dp1', 'name': 'Diamond & Pearl', 'seri...",7,Mikiko Takeda,Rare Holo,[405],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
497,neo2-1,Espeon,[Stage 1],"{'id': 'neo2', 'name': 'Neo Discovery', 'serie...",1,Atsuko Nishida,Rare Holo,[196],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...
498,ex1-1,Aggron,[Stage 2],"{'id': 'ex1', 'name': 'Ruby & Sapphire', 'seri...",1,Mitsuhiro Arita,Rare Holo,[306],{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...


In [1]:
for i in dff["set"]:
    print(i)

NameError: name 'dff' is not defined

In [40]:
import requests

# URL de la page à récupérer
url = 'https://www.tcgplayer.com/product/46466?irclickid=xqD22kwNhxyKR3Iyn6UhPXQ0UkCQkN1xmyNgTU0&sharedid=&irpid=4944541&irgwc=1&utm_source=impact&utm_medium=affiliate&utm_campaign=Scrydex&Language=English'

# Envoyer une requête GET
response = requests.get(url)

# Vérifier le statut de la requête
if response.status_code == 200:
    # Afficher le contenu HTML brut
    html_content = response.text
    print(html_content)
else:
    print(f"Erreur : {response.status_code}")


<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover" />
    <meta name="version-build" content="2970">
    <meta name="version-hash" content="2d07707b3c1ab7827d55ef6389adccb8bd975698">
    <meta name="version-name" content="origin/hotfix-cm-2911">
    <meta name="facebook-domain-verification" content="ujhbh9jn9ccwotzunt4vehh5l26w5i" />
    <script type="text/javascript">
      ;window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true},privacy:{cookies_enabled:true},ajax:{deny_list:["bam-cell.nr-data.net"]}};

      ;NREUM.loader_config={accountID:"2099792",trustKey:"2099792",agentID:"153290625",licenseKey:"a7f40de1df",applicationID:"153290625"};
      ;NREUM.info={beacon:"bam-cell.nr-data.net",errorBeacon:"bam-cell.nr-data.net",licens

In [35]:
types_uniques = df['rarity'].unique()

In [37]:
types_uniques


array(['Rare Holo', nan, 'Rare Holo GX', 'Common', 'Uncommon', 'Rare',
       'Rare Holo V', 'Promo', 'Rare Holo EX', 'Rare Ultra'], dtype=object)

In [24]:
import pandas as pd

# Exemple de données
data = [
    {'url': 'https://prices.pokemontcg.io/tcgplayer/dp3-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 5.42, 'mid': 14.49, 'high': 44.99, 'market': 14.02, 'directLow': None}, 'reverseHolofoil': {'low': 4.27, 'mid': 7.99, 'high': 12.3, 'market': 11.17, 'directLow': None}}},
    {'url': 'https://prices.pokemontcg.io/tcgplayer/ex12-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 13.02, 'mid': 17.99, 'high': 39.99, 'market': 20.76, 'directLow': 39.99}, 'reverseHolofoil': {'low': 17.0, 'mid': 32.99, 'high': 99.99, 'market': 32.49, 'directLow': 99.99}}},
    {'url': 'https://prices.pokemontcg.io/tcgplayer/mcd19-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 4.44, 'mid': 5.17, 'high': 10.26, 'market': 5.19, 'directLow': None}}},
    {'url': 'https://prices.pokemontcg.io/tcgplayer/ex7-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 14.11, 'mid': 22.39, 'high': 29.99, 'market': 24.88, 'directLow': 17.99}, 'reverseHolofoil': {'low': 15.94, 'mid': 25.0, 'high': 46.78, 'market': 15.66, 'directLow': None}}},
    {'url': 'https://prices.pokemontcg.io/tcgplayer/sm9-1', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 9.46, 'mid': 11.5, 'high': 21.49, 'market': 11.65, 'directLow': 11.33}}},
    {'url': 'https://prices.pokemontcg.io/tcgplayer/pl1-2', 'updatedAt': '2024/11/20', 'prices': {'holofoil': {'low': 8.35, 'mid': 17.82, 'high': 22.28, 'market': 20.01, 'directLow': None}, 'reverseHolofoil': {'low': 8.25, 'mid': 11.45, 'high': 25.95, 'market': 12.34, 'directLow': None}}},
]

def filter_holofoil_data(card):
    try:
        prices = card.get("prices", {})
        holofoil_price = prices.get("holofoil", {}).get("market", None)
        reverse_holofoil_price = prices.get("reverseHolofoil", {}).get("market", None)
        
        # Vérifier les conditions
        if (holofoil_price and holofoil_price > 10) or (reverse_holofoil_price and reverse_holofoil_price > 10):
            return {
                "url": card["url"],
                "holofoil": holofoil_price if holofoil_price and holofoil_price > 10 else None,
                "reverseHolofoil": reverse_holofoil_price if reverse_holofoil_price and reverse_holofoil_price > 10 else None
            }
        return None 
    except (KeyError, TypeError):
        return None

# Appliquer la fonction aux données
filtered_data = [filter_holofoil_data(card) for card in data if filter_holofoil_data(card)]

# Convertir en DataFrame pour un affichage clair
df_filtered = pd.DataFrame(filtered_data)

# Afficher le résultat
print(df_filtered)


                                             url  holofoil  reverseHolofoil
0   https://prices.pokemontcg.io/tcgplayer/dp3-1     14.02            11.17
1  https://prices.pokemontcg.io/tcgplayer/ex12-1     20.76            32.49
2   https://prices.pokemontcg.io/tcgplayer/ex7-1     24.88            15.66
3   https://prices.pokemontcg.io/tcgplayer/sm9-1     11.65              NaN
4   https://prices.pokemontcg.io/tcgplayer/pl1-2     20.01            12.34


In [26]:
filtered_data

[{'url': 'https://prices.pokemontcg.io/tcgplayer/dp3-1',
  'holofoil': 14.02,
  'reverseHolofoil': 11.17},
 {'url': 'https://prices.pokemontcg.io/tcgplayer/ex12-1',
  'holofoil': 20.76,
  'reverseHolofoil': 32.49},
 {'url': 'https://prices.pokemontcg.io/tcgplayer/ex7-1',
  'holofoil': 24.88,
  'reverseHolofoil': 15.66},
 {'url': 'https://prices.pokemontcg.io/tcgplayer/sm9-1',
  'holofoil': 11.65,
  'reverseHolofoil': None},
 {'url': 'https://prices.pokemontcg.io/tcgplayer/pl1-2',
  'holofoil': 20.01,
  'reverseHolofoil': 12.34}]

In [66]:
import pandas as pd
import numpy as np

# Fonction améliorée de filtrage des cartes avec les prix holofoil et reverseHolofoil
def filter_holofoil_data(df):
    def get_prices(x):
        if pd.isna(x["tcgplayer"]) or x["rarity"] == "Common" or pd.isna(x["rarity"]):
            return None, None, None
        
        prices = x["tcgplayer"].get("prices", {})
        holofoil_data = prices.get("holofoil", {})
        reverse_holofoil_data = prices.get("reverseHolofoil", {})

        holofoil_price = holofoil_data.get("market", None)
        reverse_holofoil_price = reverse_holofoil_data.get("market", None)

        if (holofoil_price is not None and holofoil_price > 10) or (reverse_holofoil_price is not None and reverse_holofoil_price > 10):
            return x["tcgplayer"].get("url", None), holofoil_price, reverse_holofoil_price
        else:
            return None, None, None

    df[["url", "holofoil_price", "reverse_holofoil_price"]] = df.apply(get_prices, axis=1, result_type="expand")
    
    # Supprimer les lignes où 'url' est None (pas de prix valide)
    df = df.dropna(subset=["url"])

    return df

In [67]:
x

,id,name,supertype,subtypes,level,hp,types,evolvesFrom,abilities,attacks,...,legalities,images,tcgplayer,cardmarket,evolvesTo,rules,regulationMark,url,holofoil_price,reverse_holofoil_price
0,dp3-1,Ampharos,Pokémon,[Stage 2],52,130,[Lightning],Flaaffy,"[{'name': 'Jamming', 'text': 'After your oppon...","[{'name': 'Cluster Bolt', 'cost': ['Lightning'...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/dp3/1....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,https://prices.pokemontcg.io/tcgplayer/dp3-1,14.02,11.17
1,ex12-1,Aerodactyl,Pokémon,[Stage 1],NaN,70,[Colorless],Mysterious Fossil,"[{'name': 'Reactive Protection', 'text': 'Any ...","[{'name': 'Power Blow', 'cost': ['Colorless'],...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/ex12/1...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,https://prices.pokemontcg.io/tcgplayer/ex12-1,20.76,32.49
3,ex7-1,Azumarill,Pokémon,[Stage 1],NaN,80,[Water],Marill,"[{'name': 'Froth', 'text': 'Once during your t...","[{'name': 'Water Punch', 'cost': ['Water', 'Co...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/ex7/1....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,https://prices.pokemontcg.io/tcgplayer/ex7-1,24.88,15.66
4,sm9-1,Celebi & Venusaur-GX,Pokémon,"[Basic, TAG TEAM, GX]",NaN,270,[Grass],NaN,NaN,"[{'name': 'Pollen Hazard', 'cost': ['Grass', '...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/sm9/1....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,[TAG TEAM rule: When your TAG TEAM is Knocked ...,NaN,https://prices.pokemontcg.io/tcgplayer/sm9-1,11.65,NaN
5,pl1-2,Blastoise,Pokémon,[Stage 2],60,130,[Water],Wartortle,"[{'name': 'Dig Well', 'text': 'Once during you...","[{'name': 'Aqua Press', 'cost': ['Water', 'Col...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/pl1/2....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,https://prices.pokemontcg.io/tcgplayer/pl1-2,20.01,12.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,dp3-5,Flygon,Pokémon,[Stage 2],60,120,[Fighting],Vibrava,"[{'name': 'Irritating Buzz', 'text': 'As long ...","[{'name': 'Sand Sonic', 'cost': ['Fighting', '...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/dp3/5....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,https://prices.pokemontcg.io/tcgplayer/dp3-5,10.17,5.07
220,ex8-3,Breloom,Pokémon,[Stage 1],NaN,80,[Fighting],Shroomish,NaN,"[{'name': 'Leaf Poison', 'cost': ['Colorless']...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/ex8/3....,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,NaN,NaN,https://prices.pokemontcg.io/tcgplayer/ex8-3,16.48,14.50
227,ex14-2,Blastoise δ,Pokémon,[Stage 2],NaN,110,"[Fighting, Metal]",Wartortle,"[{'name': 'Shield Veil', 'text': 'Each of your...","[{'name': 'Enraged Linear Attack', 'cost': ['F...",...,{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/ex14/2...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,NaN,[This Pokémon is both Fighting Metal type.],NaN,https://prices.pokemontcg.io/tcgplayer/ex14-2,51.70,85.76
238,sm75-3,Charizard,Pokémon,[Stage 2],NaN,160,[Fire],Charmeleon,"[{'name': 'Resolute Flame', 'text': 'This Poké...","[{'name': 'Fiery Blast', 'cost': ['Fire', 'Fir...",...,"{'unlimited': 'Legal', 'expanded': 'Legal'}",{'small': 'https://images.pokemontcg.io/sm75/3...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,{'url': 'https://prices.pokemontcg.io/cardmark...,N

In [1]:
from scraping_cleaning_data import import_data, get_prices, filter_holofoil_data

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
url = "https://api.pokemontcg.io/v2/cards"
page = 1
all_cards = []
df=import_data(url)

[{'id': 'dp3-1', 'name': 'Ampharos', 'supertype': 'Pokémon', 'subtypes': ['Stage 2'], 'level': '52', 'hp': '130', 'types': ['Lightning'], 'evolvesFrom': 'Flaaffy', 'abilities': [{'name': 'Jamming', 'text': "After your opponent plays a Supporter card from his or her hand, put 1 damage counter on each of your opponent's Pokémon. You can't use more than 1 Jamming Poké-Body each turn.", 'type': 'Poké-Body'}], 'attacks': [{'name': 'Cluster Bolt', 'cost': ['Lightning', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '70', 'text': "You may discard all Lightning Energy attached to Ampharos. If you do, this attack does 20 damage to each of your opponent's Benched Pokémon that has any Energy cards attached to it. (Don't apply Weakness and Resistance for Benched Pokémon.)"}], 'weaknesses': [{'type': 'Fighting', 'value': '+30'}], 'resistances': [{'type': 'Metal', 'value': '-20'}], 'retreatCost': ['Colorless', 'Colorless', 'Colorless'], 'convertedRetreatCost': 3, 'set': {'id': 'dp3',

In [3]:
df_clean=filter_holofoil_data(df)

AttributeError: 'list' object has no attribute 'apply'

In [4]:
df_messy=get_prices(df)

TypeError: list indices must be integers or slices, not str